In [1]:
import gc

import pandas as pd
import numpy as np

from tools.util import Util
from tools.encoding import MeanEncoder

In [2]:
df = pd.read_pickle("input/tr_test_concat.pkl")

In [3]:
df["day"] = df["click_time"].dt.day.astype('int')
df["hour"] = df["click_time"].dt.hour.astype('int')
df = df.drop(["attributed_time"], axis=1)
df = df.drop(["click_time"], axis=1)

In [4]:
columns_list = df.columns.drop(["click_id", "is_attributed", "data_set", "day", "hour"])

In [6]:
df_selected_train = df[(df["data_set"] == 0) &
                       (df["day"] == 8)]
                       
df_selected_val   = df[(df["data_set"] == 0) &
                       (df["day"] == 9) &
                       (df["hour"] >= 4) &
                       (df["hour"] <= 15)]

df_selected_all = pd.concat([df_selected_train, df_selected_val], axis=0)
df_org_test  = df[df["data_set"] == 1]

del df
gc.collect()

0

In [9]:
df_selected_train.to_pickle("input/org/selected_train.pkl")
df_selected_val.to_pickle("input/org/selected_val.pkl")
df_org_test.to_pickle("input/org/org_test.pkl")

In [62]:
df_selected_train.head()

,app,channel,click_id,data_set,device,ip,is_attributed,os,day,hour
68941878,23,153,NaN,0,1,48240,0,13,8,0
68941879,13,477,NaN,0,1,45702,0,16,8,0
68941880,12,259,NaN,0,1,123214,0,13,8,0
68941881,15,265,NaN,0,1,98052,0,19,8,0
68941882,6,125,NaN,0,1,97903,0,18,8,0


In [10]:
df_train = pd.DataFrame()
df_val   = pd.DataFrame()
df_test  = pd.DataFrame()


for columns in columns_list:
    print(columns)
    fe = FreqencyEncoder()
    fe.fit(df_selected_train[columns])
    
    df_tmp_train = fe.transform(df_selected_train[columns])
    df_train[df_tmp_train.columns] = df_tmp_train
    del df_tmp_train 
    gc.collect()
    
    
    df_tmp_val = fe.transform(df_selected_val[columns])
    df_val[df_tmp_val.columns] = df_tmp_val
    del df_tmp_val
    gc.collect()
 
    df_tmp_test = fe.transform(df_org_test[columns])
    df_test[df_tmp_test.columns] = df_tmp_test
    del df_tmp_test
    gc.collect()

app
channel
device
ip
os


In [11]:
df_test["hour"] = df_org_test["hour"].astype('int')
df_train.to_pickle("input/org/FrequencyEncoded/test.pkl")
df_test.head()

,FE_app,FE_channel,FE_device,FE_ip,FE_os,click_id,hour
0,0.080403,0.040177,0.948708,0.000005,0.015570,0,4
1,0.080403,0.016794,0.948708,0.000124,0.015570,1,4
2,0.018236,0.012704,0.948708,0.000077,0.240207,2,4
3,0.087263,0.003124,0.948708,0.000065,0.215839,3,4
4,0.133188,0.009578,0.948708,0.000001,0.215839,4,4


In [12]:
df_train["hour"] = df_selected_train["hour"].astype('int')
df_train.to_pickle("input/org/FrequencyEncoded/train.pkl")
df_train.head()

,FE_app,FE_channel,FE_device,FE_ip,FE_os,hour
68941878,0.014151,0.030456,0.948708,0.000863,0.215839,0
68941879,0.022815,0.040141,0.948708,0.000080,0.016812,0
68941880,0.133188,0.028176,0.948708,0.000013,0.215839,0
68941881,0.087263,0.030272,0.948708,0.000093,0.240207,0
68941882,0.012585,0.005584,0.948708,0.000037,0.048926,0


In [13]:

df_val.head()

,FE_app,FE_channel,FE_device,FE_ip,FE_os,hour
144708152,0.118560,0.040141,0.948708,0.000036,0.240207,4
144708153,0.053576,0.014976,0.948708,0.000011,0.048926,4
144708154,0.212623,0.109598,0.948708,0.000052,0.215839,4
144708155,0.212623,0.006714,0.948708,0.000058,0.215839,4
144708156,0.080403,0.010336,0.948708,0.000031,0.215839,4


In [14]:
X_train = df_train
y_train = df_selected_train["is_attributed"]

del df_train
gc.collect()

X_val = df_val
y_val = df_selected_val["is_attributed"]

del df_val
gc.collect()

X_test = df_test
del df_test
gc.collect()

13

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


rf = RandomForestClassifier(n_estimators=100,
                            max_depth=10,
                            n_jobs=10,
                            class_weight="balanced")
rf.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=3, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=3, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [25]:
pred_val = rf.predict(X_val)
fpr, tpr, thresholds = metrics.roc_curve(y_val, pred_val)
metrics.auc(fpr, tpr)

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


0.888300429641829

In [31]:
pred_test = rf.predict(X_test)

In [44]:
df = pd.read_pickle("input/tr_test_concat.pkl")
df_org_test  = df[df["data_set"] == 1]

In [52]:
df_org_test

,app,attributed_time,channel,click_id,click_time,data_set,device,ip,is_attributed,os
0,9,NaN,107,0.0,2017-11-10 04:00:00,1,1,5744,999,3
1,9,NaN,466,1.0,2017-11-10 04:00:00,1,1,119901,999,3
2,21,NaN,128,2.0,2017-11-10 04:00:00,1,1,72287,999,19
3,15,NaN,111,3.0,2017-11-10 04:00:00,1,1,78477,999,13
4,12,NaN,328,4.0,2017-11-10 04:00:00,1,1,123080,999,13
5,18,NaN,107,5.0,2017-11-10 04:00:00,1,1,110769,999,13
6,3,NaN,137,6.0,2017-11-10 04:00:00,1,1,12540,999,1
7,27,NaN,153,7.0,2017-11-10 04:00:00,1,1,88637,999,19
8,18,NaN,107,9.0,2017-11-10 04:00:00,1,1,14932,999,10
9,12,NaN,424,8.0,2017-11-10 04:00:00,1,1,123701,999,53


In [58]:
my_submission = pd.DataFrame()
my_submission["click_id"] = click["click_id"]
my_submission["is_attributed"] = pred_test
my_submission.to_csv("input/FE_RF_local0888300429641829.csv", index=False)

In [57]:
click.tail()

,click_id,ip,app,device,os,channel,click_time
18790464,18790464,99442,9,1,13,127,2017-11-10 15:00:00
18790465,18790465,88046,23,1,37,153,2017-11-10 15:00:00
18790466,18790467,81398,18,1,17,265,2017-11-10 15:00:00
18790467,18790466,123236,27,1,13,122,2017-11-10 15:00:00
18790468,18790468,73516,12,2,27,265,2017-11-10 15:00:00


In [ ]:
df_train = pd.DataFrame()
df_val   = pd.DataFrame()
df_test  = pd.DataFrame()

for columns in columns_list:
    print(columns)
    he = HashEncoder(n_features=5)
    he.fit(df_selected_train[columns])
    
    df_tmp_train = he.transform(df_selected_train[columns])
    df_train[df_tmp_train.columns] = df_tmp_train
    del df_tmp_train 
    gc.collect()
    
    
    df_tmp_val = he.transform(df_selected_val[columns])
    df_val[df_tmp_val.columns] = df_tmp_val
    del df_tmp_val
    gc.collect()
 
    df_tmp_test = he.transform(df_org_test[columns])
    df_test[df_tmp_test.columns] = df_tmp_test
    del df_tmp_test
    gc.collect()
    
    
df_test["click_id"] = df_org_test["click_id"].astype('int')
df_test["hour"] = df_org_test["hour"].astype('int')
df_train.to_pickle("input/org/HashEncoded/test.pkl")

df_train["hour"] = df_selected_train["hour"].astype('int')
df_train.to_pickle("input/org/HashEncoded/train.pkl")

df_val["hour"] = df_selected_val["hour"]
df_train.to_pickle("input/org/HashEncoded/val.pkl")

In [ ]:
for tr_ind, val_ind in kf.split(all_data):
    X_tr, X_val = all_data.iloc[tr_ind], all_data.iloc[val_ind]
    X_val['item_target_enc'] = X_val['item_id'].map(X_tr.groupby('item_id').target.mean())
    all_data.iloc[val_ind] = X_val

all_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data['item_target_enc'].values

In [2]:
df_selected_train = pd.read_pickle("input/org/selected_train.pkl").head(100)
df_selected_val = pd.read_pickle("input/org/selected_val.pkl").head(100)
df_org_test = pd.read_pickle("input/org/org_test.pkl").head(100)

In [ ]:
  
    
    df_val[columns] = me.transform(df_selected_val[columns])
    df_test[columns] = me.transform(df_org_test[columns])

    
df_test["click_id"] = df_org_test["click_id"].astype('int')
df_test["hour"] = df_org_test["hour"].astype('int')
print(df_test)
df_train.to_pickle("input/org/MeanEncoded/test.pkl")

df_train["hour"] = df_selected_train["hour"].astype('int')
print(df_train)
df_train.to_pickle("input/org/MeanEncoded/train.pkl")

df_val["hour"] = df_selected_val["hour"]
df_train.to_pickle("input/org/MeanEncoded/val.pkl")

In [ ]:
df_selected_train = pd.read_pickle("input/org/selected_train.pkl")
df_selected_val = pd.read_pickle("input/org/selected_val.pkl")
df_org_test = pd.read_pickle("input/org/org_test.pkl")

from sklearn.model_selection import KFold
np.random.seed(10)

kf = KFold(n_splits=3, shuffle=False)
columns_list = df_selected_train.columns.drop(["click_id", "is_attributed", "data_set", "day", "hour"])

for columns in columns_list:
    print(columns)
    dfs = []
    for tr_ind, val_ind in kf.split(df_selected_train[columns]):
        X_tr, X_val = df_selected_train[columns].iloc[tr_ind], df_selected_train[columns].iloc[val_ind]
        y_tr, y_val= df_selected_train['is_attributed'].iloc[tr_ind], df_selected_train['is_attributed'].iloc[val_ind]
        
        me = MeanEncoder()
        me.fit(X_tr, y_tr)
        me.transform(X_val)
        dfs.append(me.transform(X_val))
    
    df_selected_train["ME_" + columns] = pd.concat(dfs, axis=0)
    df_selected_val["ME_" + columns] = me.transform(df_selected_val[columns])
    df_org_test["ME_" + columns] = me.transform(df_org_test[columns])
    
df_train = df_selected_train.drop(columns_list, axis=1)
df_train = df_train.drop(["click_id", "data_set", "is_attributed"], axis=1)


df_val = df_selected_val.drop(columns_list, axis=1)
df_val = df_val.drop(["click_id", "data_set", "is_attributed"], axis=1)


df_test = df_org_test.drop(columns_list, axis=1)
df_test = df_test.drop(["click_id", "data_set", "is_attributed"], axis=1)


df_test["click_id"] = df_org_test["click_id"].astype('int')
df_test["hour"] = df_org_test["hour"].astype('int')
df_test.to_pickle("input/org/MeanEncoded/test.pkl")

df_train["hour"] = df_selected_train["hour"].astype('int')
df_train.to_pickle("input/org/MeanEncoded/train.pkl")

df_val["hour"] = df_selected_val["hour"]
df_tal.to_pickle("input/org/MeanEncoded/val.pkl")

app
channel
device
ip


In [50]:
df_test.head()

,day,hour,ME_app,ME_channel,ME_device,ME_ip,ME_os,click_id
0,10,4,0.0,0.0,0,0.0,0.0,0
1,10,4,0.0,0.0,0,0.0,0.0,1
2,10,4,0.0,0.0,0,0.0,0.0,2
3,10,4,0.0,0.0,0,0.0,0.0,3
4,10,4,0.0,0.0,0,0.0,0.0,4


In [48]:
df_org_test

,app,channel,click_id,data_set,device,ip,is_attributed,os,day,hour,ME_app,ME_channel,ME_device,ME_ip,ME_os
0,9,107,0.0,1,1,5744,999,3,10,4,0.0,0.0,0,0.0,0.0
1,9,466,1.0,1,1,119901,999,3,10,4,0.0,0.0,0,0.0,0.0
2,21,128,2.0,1,1,72287,999,19,10,4,0.0,0.0,0,0.0,0.0
3,15,111,3.0,1,1,78477,999,13,10,4,0.0,0.0,0,0.0,0.0
4,12,328,4.0,1,1,123080,999,13,10,4,0.0,0.0,0,0.0,0.0
5,18,107,5.0,1,1,110769,999,13,10,4,0.0,0.0,0,0.0,0.0
6,3,137,6.0,1,1,12540,999,1,10,4,0.0,0.0,0,0.0,0.0
7,27,153,7.0,1,1,88637,999,19,10,4,0.0,0.0,0,0.0,0.0
8,18,107,9.0,1,1,14932,999,10,10,4,0.0,0.0,0,0.0,0.0
9,12,424,8.0,1,1,123701,999,53,10,4,0.0,0.0,0,0.0,0.0
